In [1]:
# Grid InQuest II example calling GIQ dynamic library from Python.
#
# Copyright (C) 2016 Paul Michell, Michell Computing.
#
# This library is free software; you can redistribute it and/or modify it
# under the terms of the GNU Library General Public License as published by
# the Free Software Foundation; either version 2 of the License, or (at your
# option) any later version.

# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
# FITNESS FOR A PARTICULAR PURPOSE. See the GNU Library General Public License
# for more details.

# Setup required library imports.
import sys
import os
from math import radians as deg2rad
from ctypes import *
import pandas as pd
import numpy as np

In [2]:
# csv_path = r"\\mull.sms.ed.ac.uk\home\s2017658\Win7\Desktop\HareMossWaterLoggers.csv"
# csv = pd.read_csv(csv_path)
# # csv
# coord = csv[['Latitude','Longitude']].to_numpy()
# save = False

In [25]:
coord_dict = {
    'Latitude':[55.7973476,55.7973476,55.7973476],
    'Longitude':[-3.2629141,-3.2629141,-3.2629141]
}
coord_df = pd.DataFrame(coord_dict)
coord = coord_df[['Latitude','Longitude']].to_numpy()

In [26]:
# Load the GIQ dynamic library.
# if sys.platform.startswith('win'):
#   LibName = "GIQ.dll"
# elif sys.platform.startswith('linux'):
#   LibName = "libgiq.so"
# elif sys.platform.startswith('darwin'):
#   LibName = "libGIQ.dylib"
LibName = "GIQ.dll"
__file__ = 'Extensions/GIQ.dll' #r'C:\Users\s2017658\Downloads\GridInQuestII-Win64-0101e\GridInQuestII-Win64-0101e\GIQ.dll'
# Construct the GIQ library file path.
LibFolder = os.path.split(os.path.realpath(__file__))[0]
LibPath = os.path.join(LibFolder, LibName)


In [27]:
# # Load the GIQ dynamic library.
# if sys.platform.startswith('win'):
#   GIQLib = WinDLL(LibPath)
# elif sys.platform.startswith('linux'):
#   GIQLib = CDLL(LibPath)
# elif sys.platform.startswith('darwin'):
#   GIQLib = CDLL(LibPath)
# else:
#   print("Unsupported OS")
#   exit()

GIQLib = WinDLL(LibPath)

In [28]:
# Define the library coordinate structure.
class coordinates(Structure):
     _fields_ = [("x", c_double),
                 ("y", c_double),
                 ("z", c_double)]

# Reference the library convert function.
Convert = GIQLib.ConvertCoordinates
Convert.argtypes = [c_int, c_int, c_int, c_int, 
                    POINTER(coordinates),
                    POINTER(coordinates), POINTER(c_int)]
Convert.restype = bool

# Setup the calling parameter values.
SRIDSource = c_int(4937) # ETRS89 Geodetic.
RevisionSource = c_int(0) # No revision required.
SRIDTarget = c_int(27700) # OSGB36 / British National Grid
RevisionTarget = c_int(2015) # Revision for ITM/GM15.  This can also be 2002 for ITM/GM02.


In [29]:
def geod2bns(lat,long,i):
    Source = coordinates(deg2rad(long), deg2rad(lat), 0) # Longitude, Latitude, Altitude.
    Target = coordinates(0, 0, 0)
    Datum = c_int(i)
    # CallOK = bool(False)
    # Call coordinate converter.
    CallOK = Convert(SRIDSource, SRIDTarget, RevisionSource, RevisionTarget, Source, Target, Datum)
    northing = Target.y
    easting = Target.x
    return northing, easting

In [31]:
transf = np.zeros_like(coord)
for i in range(len(coord[:,0])):
    transf[i,0], transf[i,1] = geod2bns(coord[i,0],coord[i,1],i)

transf

array([[656728.51462753, 320921.48222912],
       [656728.51462753, 320921.48222912],
       [656728.51462753, 320921.48222912]])

In [25]:
csv['Northing'], csv['Easting'] = transf[:,0], transf[:,1]
# csv['northing']
csv

In [26]:
if save: csv.to_csv(csv_path)